In [16]:
import os
import cv2
import numpy as np
import time

In [17]:
def remap_image_1(image_path, output_path, rad=np.pi/4):
    img = cv2.imread(image_path)
    if img is None:
        raise FileNotFoundError(f"The specified image file at {image_path} could not be loaded.")
    
    h, w = img.shape[:2]
    w_half = int(w / 2)
    h_half = int(h / 2)

    phi, theta = np.meshgrid(np.linspace(-np.pi, np.pi, w_half*2),
                             np.linspace(-np.pi/2, np.pi/2, h_half*2))

    x = np.cos(theta) * np.cos(phi)
    y = np.cos(theta) * np.sin(phi)
    z = np.sin(theta)

    # ピッチ周りに π/2 だけ回転
    rot = rad
    xx = x * np.cos(rot) - y * np.sin(rot)
    yy = x * np.sin(rot) + y * np.cos(rot)
    zz = z

    theta = np.arcsin(zz)
    phi = np.arctan2(yy, xx)

    Y = 2 * theta / np.pi * h_half + h_half
    X = phi / np.pi * w_half + w_half

    out = cv2.remap(img, X.astype(np.float32), Y.astype(np.float32), cv2.INTER_LINEAR, borderMode=cv2.BORDER_WRAP)
    cv2.imwrite(output_path, out)

In [18]:
def remap_image_2(image_path, output_path, rad=np.pi/4):
    img = cv2.imread(image_path)
    if img is None:
        raise FileNotFoundError(f"The specified image file at {image_path} could not be loaded.")
    
    h, w = img.shape[:2]
    w_half = int(w / 2)
    h_half = int(h / 2)

    phi, theta = np.meshgrid(np.linspace(-np.pi, np.pi, w_half*2),
                             np.linspace(-np.pi/2, np.pi/2, h_half*2))

    x = np.cos(theta) * np.cos(phi)
    y = np.cos(theta) * np.sin(phi)
    z = np.sin(theta)

    rot = rad
    xx = x * np.cos(rot) + z * np.sin(rot)
    yy = y
    zz = -x * np.sin(rot) + z * np.cos(rot)
    
    theta = np.arcsin(zz)
    phi = np.arctan2(yy, xx)

    Y = 2 * theta / np.pi * h_half + h_half
    X = phi / np.pi * w_half + w_half

    out = cv2.remap(img, X.astype(np.float32), Y.astype(np.float32), cv2.INTER_LINEAR, borderMode=cv2.BORDER_WRAP)
    cv2.imwrite(output_path, out)

In [19]:
def remap_image_3(image_path, output_path, rad=np.pi/4):
    img = cv2.imread(image_path)
    if img is None:
        raise FileNotFoundError(f"The specified image file at {image_path} could not be loaded.")
    
    h, w = img.shape[:2]
    w_half = int(w / 2)
    h_half = int(h / 2)

    phi, theta = np.meshgrid(np.linspace(-np.pi, np.pi, w_half*2),
                             np.linspace(-np.pi/2, np.pi/2, h_half*2))

    x = np.cos(theta) * np.cos(phi)
    y = np.cos(theta) * np.sin(phi)
    z = np.sin(theta)

    rot = rad
    xx = x
    yy = y * np.cos(rot) - z * np.sin(rot)
    zz = y * np.sin(rot) + z * np.cos(rot)
    theta = np.arcsin(zz)
    phi = np.arctan2(yy, xx)

    Y = 2 * theta / np.pi * h_half + h_half
    X = phi / np.pi * w_half + w_half

    out = cv2.remap(img, X.astype(np.float32), Y.astype(np.float32), cv2.INTER_LINEAR, borderMode=cv2.BORDER_WRAP)
    cv2.imwrite(output_path, out)

In [42]:
def rotate_yaw(x, y, z, rot):
    xx = x * np.cos(rot) - y * np.sin(rot)
    yy = x * np.sin(rot) + y * np.cos(rot)
    zz = z

    return xx, yy, zz

def rotate_roll(x, y, z, rot):
    xx = x
    yy = y * np.cos(rot) - z * np.sin(rot)
    zz = y * np.sin(rot) + z * np.cos(rot)

    return xx, yy, zz

def rotate_pitch(x, y, z, rot):
    xx = x * np.cos(rot) + z * np.sin(rot)
    yy = y
    zz = -x * np.sin(rot) + z * np.cos(rot)

    return xx, yy, zz

In [43]:
def remap_image_proposed(image_path, output_path, rad=np.pi/2):
    t_b = time.perf_counter()
    img = cv2.imread(image_path)
    if img is None:
        raise FileNotFoundError(f"The specified image file at {image_path} could not be loaded.")
    
    h, w = img.shape[:2]
    w_half = int(w / 2)
    h_half = int(h / 2)

    phi, theta = np.meshgrid(np.linspace(-np.pi, np.pi, w_half*2),
                             np.linspace(-np.pi/2, np.pi/2, h_half*2))

    x = np.cos(theta) * np.cos(phi)
    y = np.cos(theta) * np.sin(phi)
    z = np.sin(theta)

    xx, yy, zz = rotate_yaw(x, y, z, rad)
    xx, yy, zz = rotate_pitch(xx, yy, zz, rad)
    xx, yy, zz = rotate_yaw(xx, yy, zz, rad)

    theta = np.arcsin(zz)
    phi = np.arctan2(yy, xx)

    Y = 2 * theta / np.pi * h_half + h_half
    X = phi / np.pi * w_half + w_half

    t_b = time.perf_counter()
    img = cv2.imread(image_path)
    out = cv2.remap(img, X.astype(np.float32), Y.astype(np.float32), cv2.INTER_LINEAR, borderMode=cv2.BORDER_WRAP)
    t_a = time.perf_counter()
    print(t_a - t_b)
    cv2.imwrite(output_path, out)
    
remap_image_proposed('./data/2.png', 'proposed.png', np.pi/2)

0.0110201999777928


In [46]:
def make_image_map(img_hw, rad=np.pi/2):
    (h, w) = img_hw
    w_half = int(w / 2)
    h_half = int(h / 2)

    phi, theta = np.meshgrid(np.linspace(-np.pi, np.pi, w_half*2),
                             np.linspace(-np.pi/2, np.pi/2, h_half*2))

    x = np.cos(theta) * np.cos(phi)
    y = np.cos(theta) * np.sin(phi)
    z = np.sin(theta)

    xx, yy, zz = rotate_yaw(x, y, z, rad)
    xx, yy, zz = rotate_pitch(xx, yy, zz, rad)
    xx, yy, zz = rotate_yaw(xx, yy, zz, rad)

    theta = np.arcsin(zz)
    phi = np.arctan2(yy, xx)

    Y_remap = 2 * theta / np.pi * h_half + h_half
    X_remap = phi / np.pi * w_half + w_half

    return Y_remap, X_remap


img_sample = cv2.imread('./data/2.png')
img_hw = img_sample.shape[:2]
Y_remap, X_remap = make_image_map(img_hw)

In [47]:
def remap_image(image_path, output_path, YX_remap):
    (Y_remap, X_remap) = YX_remap
    img = cv2.imread(image_path)
    t_b = time.perf_counter()
    out = cv2.remap(img, X_remap.astype(np.float32), Y_remap.astype(np.float32), 
                    cv2.INTER_LINEAR, borderMode=cv2.BORDER_WRAP)
    t_a = time.perf_counter()
    ret = t_a - t_b
    cv2.imwrite(output_path, out)

    return ret
    
    
t = remap_image('./data/2.png', 'proposed2.png', (Y_remap, X_remap))
print(t)

0.0023044999688863754


In [23]:
def remap_image_proposed2(image_path, output_path, rad=np.pi/2):
    img = cv2.imread(image_path)
    t_b = time.perf_counter()
    if img is None:
        raise FileNotFoundError(f"The specified image file at {image_path} could not be loaded.")
    
    h, w = img.shape[:2]
    w_half = int(w / 2)
    h_half = int(h / 2)

    phi, theta = np.meshgrid(np.linspace(-np.pi, np.pi, w_half*2),
                             np.linspace(-np.pi/2, np.pi/2, h_half*2))

    x = np.cos(theta) * np.cos(phi)
    y = np.cos(theta) * np.sin(phi)
    z = np.sin(theta)

    R_combined = np.array([
        [-1, 0, 0],
        [0, 0, 1],
        [0, 1, 0]
    ])

    # Apply the rotation matrix
    xyz = np.stack([x, y, z], axis=-1)
    xyz_rotated = np.einsum('ij, ...j', R_combined, xyz)

    # Convert back to spherical coordinates
    xx, yy, zz = xyz_rotated[..., 0], xyz_rotated[..., 1], xyz_rotated[..., 2]

    theta = np.arcsin(zz)
    phi = np.arctan2(yy, xx)

    Y = 2 * theta / np.pi * h_half + h_half
    X = phi / np.pi * w_half + w_half

    t_b = time.perf_counter()
    img = cv2.imread(image_path)
    out = cv2.remap(img, X.astype(np.float32), Y.astype(np.float32), cv2.INTER_LINEAR, borderMode=cv2.BORDER_WRAP)
    t_a = time.perf_counter()
    print(t_a - t_b)
    cv2.imwrite(output_path, out)
    
remap_image_proposed2('./data/2.png', 'proposed2.png', np.pi/2)

0.010976299992762506


In [7]:
remap_image_1('./data/2.png', 'a.png', np.pi/2)
remap_image_2('./data/2.png', 'b.png', np.pi/2)
remap_image_3('./data/2.png', 'c.png', np.pi/2)

In [8]:
remap_image_2('a.png', 'ab.png', np.pi/2)

In [52]:
remap_image_1('ab.png', 'aba.png', np.pi/2)